In [2]:
#Cargar bibliotecas necesarias y especificar ruta de trabajo.
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
import os
import pandas as pd
import glob
import zipfile
import requests


print("¿Desea introducir una ruta de trabajo? (Escriba si o no)")
print("si selecciona NO el programa se ejecurá donde este guardado este fichero.")
selectRuta = input()
selectRuta = selectRuta.lower()

if selectRuta == "si" or selectRuta == "s" or selectRuta == "yes" or selectRuta == "y":
    print("introduzca la ruta deseada: ")
    path = input()
    print("Ruta: " + path)
elif selectRuta == "no" or selectRuta == "n":
    path = os.getcwd()
    print("Ruta: " + path)

¿Desea introducir una ruta de trabajo? (Escriba si o no)
si selecciona NO el programa se ejecurá donde este guardado este fichero.


 n


Ruta: C:\Users\Alberto\Anaconda Python


In [3]:
def concatCSVs(path, tipoArchivo, output):
    df = pd.DataFrame()
    file_list = []
    all_files = glob.glob(path + tipoArchivo + ".csv")
    for f in all_files:
        data = pd.read_csv(f, delimiter=';')
        nombre = os.path.basename(f)
        data['source_file'] = nombre
        file_list.append(data)
    df = pd.concat(file_list)
    df.to_csv(path + output, encoding="utf-8")  #--------------->Mirar el formato: 
                              #DataFrame.to_csv(path_or_buf=None, sep=',', na_rep='', float_format=None, columns=None, header=True, 
                              #index=True, index_label=None, mode='w', encoding=None, compression='infer', quoting=None, quotechar='"', 
                              #line_terminator=None, chunksize=None, date_format=None, doublequote=True, escapechar=None, decimal='.', 
                              #errors='strict', storage_options=None)

def concatCSVsTRF(path, tipoArchivo, output):
    df = pd.DataFrame()
    file_list = []
    all_files = glob.glob(path+ tipoArchivo +".csv")
    for f in all_files:
        data = pd.read_csv(f, delimiter=';',encoding='unicode_escape') #encoding='unicode_escape'
        nombre = os.path.basename(f)
        data['source_file'] = nombre
        file_list.append(data)
    df = pd.concat(file_list, sort=True)
    df.to_csv(path + output, encoding="utf-8") #--------------->Mirar el formato: 
                              #DataFrame.to_csv(path_or_buf=None, sep=',', na_rep='', float_format=None, columns=None, header=True, 
                              #index=True, index_label=None, mode='w', encoding=None, compression='infer', quoting=None, quotechar='"', 
                              #line_terminator=None, chunksize=None, date_format=None, doublequote=True, escapechar=None, decimal='.', 
                              #errors='strict', storage_options=None)
    
def descompresionDeArchivosCSV(path):
    all_files = glob.glob(path+"ACA_*")
    for archivo in all_files:
        z = zipfile.ZipFile(archivo, "r")
        z.extractall(path)
        z.close()
        
def limpiezeDeArchivos(path, tipo): #path indica la rita u tipo el archivo
    all_files = glob.glob(path + tipo)
    for archivo in all_files:
        os.remove(archivo)

### Descarga de ficheros meteorológicos:
Algoritmo de scraping para la web del ayuntamiento de Madrid para la obtención de documentos csv 
cuyo contenidos son los datos meteorológicos diarios desde 2019 hasta 2021 (aun sin concluir).

Pagina donde se extraen los datos: 
https://datos.madrid.es/sites/v/index.jsp?vgnextoid=fa8357cec5efa610VgnVCM1000001d4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD

In [4]:
# crear la carpeta de Meteorologáa, se pone exist_ok por si existe que no de error
os.makedirs(path + "/Meteorologia", exist_ok=True)

# indicar la ruta Archivos Meteorologicos.
url_page = 'https://datos.madrid.es/sites/v/index.jsp?vgnextoid=fa8357cec5efa610VgnVCM1000001d4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD'
fileType = '.csv'
domain = 'https://datos.madrid.es/'
page = requests.get(url_page).text 
soup = BeautifulSoup(page, "lxml")

#se generan los archivos meteorológicos.
i = 0
for link in soup('a'):
        link_file = link.get('href')
        if None != link_file and fileType in link_file:
            nombre = "METEO_" + str(i) + ".csv"
            with open(path + "/Meteorologia/" + nombre, 'wb') as file:
                response = requests.get(domain + link_file)
                file.write(response.content)
            i+=1
                      
# Unión de todos los .csv en uno solo.
output = "ArchivoMeteorologia.csv"
concatCSVs(path+"/Meteorologia/", "METEO_*",output) #funcion para concatenar todos los archivos csv del clima en un inico archivo.

### Descarga de ficheros del tráfico:
Algoritmo de scraping para la web del ayuntamiento de Madrid para la obtención de documentos csv 
cuyo contenidos son los datos del trafico diarios desde 2019 hasta 2021 (aun sin concluir).

Página donde se extraen los datos: 
https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=fabbf3e1de124610VgnVCM2000001f4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default

In [5]:
# crear la carpeta de Meteorologáa, se pone exist_ok por si existe que no de error
os.makedirs(path + "/Trafico", exist_ok=True)

# indicar la ruta
url_page = 'https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=fabbf3e1de124610VgnVCM2000001f4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default'
fileType = '.csv'
domain = 'https://datos.madrid.es/'
page = requests.get(url_page).text 
soup = BeautifulSoup(page, "lxml")

#se genera los archivos tráfico
i = 33 #12 * 2 (2019-2020) + 9 (de 2021)
for link in soup('a'):
    if i > 0: #--------------------------> para extraer solo hasta el 2019.
        link_file = link.get('href')
        if None != link_file and fileType in link_file:
            nombre = "TRF_" + str(i) + ".csv"
            with open(path + "/Trafico/" + nombre, 'wb') as file:
                response = requests.get(domain + link_file)
                file.write(response.content)
            i-=1
            
# Unión de todos los .csv en uno solo.
output = "ArchivoTrafico.csv"
concatCSVsTRF(path+"/Trafico/", "TRF_*",output) #funcion para concatenar todos los archivos csv de trafico en un único archivo.

### Descarga de ficheros de contaminación:
Pagina donde se obtienen los datos: https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=f3c0f7d512273410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default

In [6]:
# crear la carpeta de Meteorologáa, se pone exist_ok por si existe que no de error
os.makedirs(path + "/Contaminacion", exist_ok=True)

# indicar la ruta
url_page = 'https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=f3c0f7d512273410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default'
fileType = '.zip'
page = requests.get(url_page).text 
soup = BeautifulSoup(page, "lxml")

#se genera los archivos ACA... (Archivo de Calidad del Aire)
import datetime
i = datetime.date.today().year
for link in soup('a'):
    if i > 2018: #--------------------------> para extraer solo hasta el 2019.
        link_file = link.get('href')
        if None != link_file and fileType in link_file:
            nombre = "ACA_" + str(i) + ".zip"
            with open(path + "/Contaminacion/" + nombre, 'wb') as file:
                response = requests.get(domain + link_file)
                file.write(response.content)
            i-=1
            
#Descompresión de los archivos comprimidos descargados
descompresionDeArchivosCSV(path+"/Contaminacion/")
tipos = [".xml", ".txt",".zip"]
for tipo in tipos:
    limpiezeDeArchivos(path+"/Contaminacion/*", tipo)
    
#Unión de todos los .csv en uno solo.   
output = "ArchivoContaminacion.csv"
concatCSVs(path+"/Contaminacion/", "*",output) #funcion para concatenar todos los archivos csv de calidad del aire en un único archivo.

### Descarga de informacion localizacion de estaciones de calidad del Aire y Meterorologícas.

###### Estaciones de control Contaminación: https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=9e42c176313eb410VgnVCM1000000b205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default
###### Estaciones de control Climatología: https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=2ac5be53b4d2b610VgnVCM2000001f4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default

In [7]:
#Código para estaciones de contaminación
url = "https://datos.madrid.es/egob/catalogo/212629-1-estaciones-control-aire.csv"
response = requests.get(url)

# lo grabamos en formato csv
f = open(path+"/Contaminacion/"+'EstacionesContaminacion.csv', 'wb')
f.write(response.content)
f.close()

#Código para estaciones meteorológicas
url = "https://datos.madrid.es/egob/catalogo/300360-1-meteorologicos-estaciones.csv"
response = requests.get(url)

# lo grabamos en formato csv
f = open(path+"/Meteorologia/"+'EstacionesMeteorologicas.csv', 'wb')
f.write(response.content)
f.close()